**Prophet** is a tool (made by Facebook) for predicting time-series. 
I was exploring it's capabilities for this competition, and will share this small starter script - maybe somebody will find something useful here.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from fbprophet import Prophet
from sklearn.metrics import mean_squared_log_error

In [ ]:
STORE = 1
ITEM = 105575
print('Reading data for store {}, item {}...'.format(STORE, ITEM))
it = pd.read_csv('../input/train.csv', iterator=True, chunksize=10000)
df = pd.concat([c[(c['store_nbr'] == STORE) & (c['item_nbr'] == ITEM)] for c in it])
print('Time-series data shape: {}'.format(df.shape))

In [ ]:
TRAIN_SIZE = 365
CV_SIZE = 16 #if you make it bigger, fill missing dates in cv with 0 if any
X = df[-(TRAIN_SIZE+CV_SIZE):-CV_SIZE]
y = df[-CV_SIZE:]
print('Train on: {}, CV: {}'.format(X.shape, y.shape))

X = X[['date','unit_sales']]
X.columns = ['ds', 'y'] #Prophet names
print(X.tail())

In [ ]:
m = Prophet(yearly_seasonality=True)
m.fit(X)
print('Prophet fitted')

In [ ]:
future = m.make_future_dataframe(periods=CV_SIZE)
pred = m.predict(future)
print(pred[['ds','yhat','yhat_lower','yhat_upper']].tail(5))
m.plot(pred)

Looks like Prophet noticed some weekly seasonality in data and also managed to include it in predictions. Python API allows to extract different components of predicions to check it...

In [ ]:
m.plot_components(pred)

So we can see that overall trend, yearly and weekly seasonality will influence predictions for this item. This looks like not typical item/store, because typically saturdays and sundays are top days, but here sundays are with lower popularity. But still we can see some spike around Christmas time in yearly graph. Let's try to score predictions to see, how good this tool can be...

In [ ]:
pred['ds'] = pred['ds'].astype(str)
data = pred[['ds','yhat']].merge(y, left_on='ds', right_on='date')

items = pd.read_csv('../input/items.csv') #we need items for weights
items['weight'] = 1 + items['perishable'] * 0.25
data = data.merge(items[['item_nbr','weight']], how='left', on='item_nbr')

score = np.sqrt(mean_squared_log_error(data['unit_sales'].clip(0, 999999), data['yhat'].fillna(0).clip(0, 999999), data['weight']))
print('Score:{}'.format(score))

Score 0.5676 - not bad, if we are using only time-series data here (no *onpromotion*, etc important features). 

Just to note - scores differs quite a lot for different items/stores, so not all of them are so good.

Enjoy.